# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# identify file path
weather_csv = 'weather_data.csv'

# turn csv into dataframe
weather_df = pd.read_csv(weather_csv)

weather_df.head()

,Name,Country,Latitude,Longitude,Temperature(F),Wind Speed(mph),Cloud Coverage,Humidity
0,Manhattan,US,40.7834,-73.9662,44.870,3.09,1,24
1,Iquitos,PE,-3.7481,-73.2472,86.270,4.12,40,70
2,Berlevåg,NO,70.8578,29.0864,21.470,6.17,75,86
3,Albany,US,42.6001,-73.9662,38.264,1.25,28,28
4,Mataura,NZ,-46.1927,168.8643,59.270,1.34,66,75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
#change g_key to api_key
gmaps.configure(api_key=g_key)

# convert humidity to float, drop NaN values
weather_df = weather_df.dropna()
humidity = weather_df['Humidity'].astype(float)

# store lattitude and longitude together
local = weather_df[['Latitude', 'Longitude']].astype(float)

local = local.to_records(index=False)
local = list(local)



# create humidity heatmap layer

m = gmaps.figure()
m.add_layer(gmaps.heatmap_layer(local, weights = humidity))
m

# m = gmaps.Map()

# humidity_layer = m.WeightedHeatmap(data=locations)

# m.add_layer(humidity_layer)
# m



Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
# copy weather_df
best_weather_df = weather_df.copy()
best_weather_df = best_weather_df.dropna()

# filter through df to find cities with ideal conditions
best_weather_df = best_weather_df.loc[(best_weather_df['Temperature(F)'] < 75) & (best_weather_df['Temperature(F)'] > 65)]
best_weather_df = best_weather_df.loc[(best_weather_df['Wind Speed(mph)'] < 10)]
best_weather_df = best_weather_df.loc[(best_weather_df['Cloud Coverage'] < 50) & (best_weather_df['Cloud Coverage'] > 10)]

best_weather_df.head(10)

,Name,Country,Latitude,Longitude,Temperature(F),Wind Speed(mph),Cloud Coverage,Humidity
13,Menongue,AO,-14.6585,17.6910,65.948,0.99,26,68
30,Torres,BR,-29.3353,-49.7269,73.976,0.32,45,86
31,Davila,PH,18.4726,120.5763,73.670,2.06,20,83
120,Qui Nhon,VN,13.7667,109.2333,72.068,2.94,40,90
158,Castro,BR,-24.7911,-50.0119,66.398,0.98,37,88
161,Kiama,AU,-34.6833,150.8667,70.268,0.45,15,82
188,Gamboula,CF,4.1178,15.1393,74.480,1.00,47,55
197,Thohoyandou,ZA,-22.9456,30.4850,71.420,1.99,17,73
224,São Filipe,CV,14.8961,-24.4956,69.800,5.61,14,75
228,Kalakamati,BW,-20.5929,27.3121,69.980,2.11,34,55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df = best_weather_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
row_index = 0

# limit hotel search within 5000m of each location
params = {'radius': 5000, 
          'types': 'lodging',
          'key': g_key}

# walk through hotel_df
for i, row in hotel_df.iterrows():
        
    # grab lat and lng data
    lat = row['Latitude']
    lng = row['Longitude']
    
    # add lat and lng data to params dict
    params['location'] = f'{lat}, {lng}'
    
    # API request
    print(f'Request number {row_index} for {row["Name"]}.')
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    
    try:
        print(f"Closest hotel: {results[0]['name']}")
        hotel_df.loc[row_index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print(' ')
    
    row_index = row_index + 1

Request number 0 for Menongue.
Closest hotel: Ritz Lauca
 
Request number 1 for Torres.
Closest hotel: Farol Hotel
 
Request number 2 for Davila.
Closest hotel: Blue Ridge Beach Resort
 
Request number 3 for Qui Nhon.
Closest hotel: Hoang Yen Canary Hotel
 
Request number 4 for Castro.
Closest hotel: CHACARA BAILLY
 
Request number 5 for Kiama.
Closest hotel: The Sebel Kiama Harbourside
 
Request number 6 for Gamboula.
Missing field/result... skipping.
 
Request number 7 for Thohoyandou.
Closest hotel: Masili Guesthouse & Conference
 
Request number 8 for São Filipe.
Closest hotel: Tortuga B&B
 
Request number 9 for Kalakamati.
Missing field/result... skipping.
 
Request number 10 for Bud.
Closest hotel: Ted McQuinn Trust
 
Request number 11 for Mosquera.
Closest hotel: Hotel SJ
 
Request number 12 for Sibu.
Closest hotel: RH HOTEL (4 Star)
 
Request number 13 for Mar del Plata.
Closest hotel: Gran Hotel Mar del Plata
 
Request number 14 for Coquimbo.
Closest hotel: Punto de encuentro


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
locations = locations.head(28)
locations

,Latitude,Longitude
13,-14.6585,17.6910
30,-29.3353,-49.7269
31,18.4726,120.5763
120,13.7667,109.2333
158,-24.7911,-50.0119
161,-34.6833,150.8667
188,4.1178,15.1393
197,-22.9456,30.4850
224,14.8961,-24.4956
228,-20.5929,27.3121


In [51]:
# Add marker layer ontop of heat map
symbol = gmaps.symbol_layer(locations, fill_color='red', stroke_color='red')

m.add_layer(symbol)
# Display figure
m

Figure(layout=FigureLayout(height='420px'))